In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import deepRD
from deepRD.diffusionIntegrators import langevinNoiseSampler
from deepRD.noiseSampler import noiseSampler
from deepRD.noiseSampler import binnedData_qi, binnedData_ri

In [ ]:
localDataDirectory = './data/'

In [ ]:
# Load binning sampling models
binnedDataFilename = localDataDirectory + 'qiBinnedData.pickle'
#binnedDataFilename = localDataDirectory + 'riBinnedData.pickle'
binnedData = pickle.load(open(binnedDataFilename, "rb" ))
parameters = binnedData.parameterDictionary
print(parameters)

In [ ]:
# Define particle list
D = parameters['D']
position = [0,0,0]
velocity = [0,0,0]
mass =  parameters['mass']
particle1 = deepRD.particle(position, D, velocity, mass)
particleList = deepRD.particleList([particle1])

In [ ]:
# Define noise sampler
nSampler = noiseSampler(binnedData)

In [ ]:
# Define integrator
dt = parameters['dt']  
stride = 1
tfinal = 1000
KbT = parameters['KbT']
boxsize = 1 #parameters['boxsize']
diffIntegrator = langevinNoiseSampler(dt, stride, tfinal, nSampler, KbT, boxsize)

In [ ]:
t, X, V= diffIntegrator.propagate(particleList)

In [ ]:
# Plot trajectory in x, y and z
particleIndex = 0
#plt.plot(X[:,particleIndex], label = ['x','y','z']);
xpos = X[:,particleIndex,0]
ypos = X[:,particleIndex,1]
zpos = X[:,particleIndex,2]
plt.plot(t,zpos, 'o', label = 'x');
plt.legend()

In [ ]:
numbins=25
#averagepos = (np.array(xpos)+np.array(ypos)+np.array(zpos))/3.0
#plt.hist(averagepos, bins = numbins, density= True, alpha =0.3)
plt.hist(xpos, bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(ypos, bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(zpos, bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
plt.xlim([-0.15,0.15])
plt.show()

In [ ]:
plt.hist(xpos, bins = numbins, density= True, alpha=0.5, label='x')
